In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models, Input
import pandas as pd
import numpy as np
from PIL import Image
import os
import glob
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
def load_data(image_dir, labels_file):
    data = pd.read_excel(labels_file)
    images = []
    labels = []

    for img_path in glob.glob(os.path.join(image_dir, "*.png")):
        try:
            img = Image.open(img_path).convert('L')
            img = img.resize((200, 50))
            img_array = np.array(img) / 255.0
            img_num = int(os.path.basename(img_path).split('.')[0])
            answer = data.iloc[img_num-1]['Answer']
            images.append(img_array)
            labels.append(float(answer))
        except Exception as e:
            print(f"Skipping {img_path}: {e}")

    return np.array(images), np.array(labels)

In [3]:
def create_model():

    # Входной слой ожидает изображения размером 50x200 пикселей с 1 черно-белым каналом
    inputs = Input(shape=(50, 200, 1))

    # Первый сверточный блок
    x = layers.Conv2D(64, (3, 3), padding='same')(inputs) # Применяем 64 фильтра размером для обнаружения базовых признаков (грани)
    x = layers.BatchNormalization()(x) # Нормализуем для стабильности
    x = layers.Activation('relu')(x) # Функция активации
    x = layers.MaxPooling2D((2, 2))(x) # Уменьшаем пространственные размеры, сохраняя самые сильные признаки

    # В остальных блоках плюс-минус то же самое

    # Второй сверточный блок
    x = layers.Conv2D(128, (3, 3), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.MaxPooling2D((2, 2))(x)

    # Третий сверточный блок
    x = layers.Conv2D(256, (3, 3), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.MaxPooling2D((2, 2))(x)

    x = layers.Flatten()(x) # Преобразование 3д в 1д для полносвязных слоев
    x = layers.Dense(512, activation='relu')(x) # Первый полносвязный слой
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.5)(x) # Дропаут, чтобы не было переобучения
    x = layers.Dense(256, activation='relu')(x) # Второй полносвязный слой
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(1)(x) # Выходной слой

    return models.Model(inputs=inputs, outputs=outputs)

In [4]:
def train_model(image_dir, labels_file):
    X, y = load_data(image_dir, labels_file)
    X = X.reshape(-1, 50, 200, 1)

    split = int(0.8 * len(X))
    X_train, X_test = X[:split], X[split:]
    y_train, y_test = y[:split], y[split:]

    # Аугментация выборки
    datagen = ImageDataGenerator(
        rotation_range=5,
        width_shift_range=0.1,
        height_shift_range=0.1,
        zoom_range=0.1,
        shear_range=0.1
    )

    model = create_model()
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
                 loss='mse',
                 metrics=['mae'])

    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6
    )

    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss', patience=10, restore_best_weights=True
    )

    history = model.fit(
        datagen.flow(X_train, y_train, batch_size=32),
        epochs=50,
        validation_data=(X_test, y_test),
        callbacks=[reduce_lr, early_stopping],
        verbose=1
    )

    test_loss, test_mae = model.evaluate(X_test, y_test, verbose=0)
    print(f"\nTest MAE: {test_mae:.2f}")

    return model, history

In [5]:
image_dir = "/content/drive/MyDrive/repos/Handwritten_equations_images"
labels_file = "/content/Maths_eqations_handwritten.xlsx"
model, history = train_model(image_dir, labels_file)

Skipping /content/drive/MyDrive/repos/Handwritten_equations_images/60.png: single positional indexer is out-of-bounds
Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


2/2 ━━━━━━━━━━━━━━━━━━━━ 9s 3s/step - loss: 28612.9707 - mae: 72.2758 - val_loss: 844.9903 - val_mae: 20.7718 - learning_rate: 1.0000e-04
Epoch 2/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 4s/step - loss: 17745.5645 - mae: 57.9718 - val_loss: 844.8584 - val_mae: 20.7673 - learning_rate: 1.0000e-04
Epoch 3/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 2s/step - loss: 22588.9609 - mae: 67.4229 - val_loss: 842.5577 - val_mae: 20.7110 - learning_rate: 1.0000e-04
Epoch 4/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 9s 2s/step - loss: 20620.4824 - mae: 62.0896 - val_loss: 838.7894 - val_mae: 20.6199 - learning_rate: 1.0000e-04
Epoch 5/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 3s/step - loss: 20947.8223 - mae: 69.9201 - val_loss: 834.8255 - val_mae: 20.5234 - learning_rate: 1.0000e-04
Epoch 6/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 9s 3s/step - loss: 20680.1016 - mae: 68.3666 - val_loss: 830.5815 - val_mae: 20.4197 - learning_rate: 1.0000e-04
Epoch 7/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 2s/step - loss: 16124.6348 - mae: 57.8570 - val_loss: 826.6328 - val_mae: 20.3